In [0]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Conv2D, Input, Lambda, Flatten, Reshape, MaxPool2D, Multiply, Activation, Add, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
import h5py
import matplotlib.pyplot as plt
from google.colab import drive

In [2]:
from numpy.random import seed
seed(1)
print(tf.__version__)
tf.random.set_seed(0)

2.2.0-rc2


In [0]:
physical_devices = tf.config.list_physical_devices('GPU') 
assert len(physical_devices) > 0

In [4]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd "drive/My Drive/BTP/code"
%ls

/content/drive/My Drive/BTP/code
Attention_train_v1.ipynb       extra/
Attention_train_v1-LTPA.ipynb  __init__.py
Attention_train_v2.ipynb       multi-head_attention_mnist_v1.ipynb
Attention_train_v2-LTPA.ipynb  multi-head_attention_mnist_v2.ipynb
Bateaux_code/                  multi-head_attention_mnist_v3.ipynb
bateaux_metrics_v1.py          playground.ipynb
bateaux_metrics_v2.py          __pycache__/
dataset_creation/              siamese.ipynb
evaluate_models.ipynb          Untitled0.ipynb


In [6]:
from bateaux_metrics_v2 import *

Using TensorFlow backend.


In [0]:
parent_dir = os.path.dirname(os.getcwd())
train_set_file = os.path.join(parent_dir, 'sets_labels', 'combined', 'shuffled', 'train_set.hdf5')
train_GT_file = os.path.join(parent_dir, 'sets_labels', 'combined', 'shuffled', 'train_labels.hdf5')
test_set_file = os.path.join(parent_dir, 'sets_labels', 'combined', 'shuffled', 'test_set.hdf5')
test_GT_file = os.path.join(parent_dir, 'sets_labels', 'combined', 'shuffled', 'test_labels.csv')

weight_file = os.path.join(parent_dir, 'models', 'Attention', '10-scenes', 'att_model_10-scenes_v3.hdf5')
info_file = os.path.join(parent_dir, 'models', 'Attention', '10-scenes', 'att_model_10-scenes_v3.txt')

In [0]:
tf.keras.backend.clear_session()

In [0]:
img_width = 128
img_height = 128
channels = 1

In [0]:
def main_attention(x, l, d, dv, nv):  #x, l=6*6, d=64*3 , dv=8*3, nv = 8
    v = Dense(d, activation = 'relu')(x)
    k = Dense(d, activation = 'relu')(x)
    q = Dense(d, activation = 'relu')(x)
   
    v = Reshape([l,nv,dv])(v)  
    k = Reshape([l,nv,dv])(k)  
    q = Reshape([l,nv,dv])(q)
   
    new = Multiply()([q,k])
    new = Activation('softmax')(new)
   
    new1 = Multiply()([new,v])
    new1 = Reshape([l,d])(new1)
    out = Add()([new1,x])
   
    return out

In [0]:
def multi_head(l, d, dv, nv): #l=6*6, d=64*3 , dv=8*3, nv = 8
   
    input_img = Input(shape = (l,d))
   
    out_1 = main_attention(input_img, l=l, d=d, dv=dv, nv=nv)
    out_2 = main_attention(input_img, l=l, d=d, dv=dv, nv=nv)
    out = Concatenate(axis = 2)([out_1,out_2])
    att_model = Model(inputs = input_img, outputs = out)
    #print(att_model.summary())
    return att_model

In [12]:
inp = Input((img_width, img_height, channels), name='input_layer')
x = Conv2D(64, (3,3), padding='same')(inp)
#x = Conv2D(64, (3,3),activation='relu', padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3,3), padding='same')(x)
#x = Conv2D(128, (3,3),activation='relu', padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = Conv2D(256, (3,3),activation='relu', padding='same')(x)
#x = Conv2D(256, (3,3),activation='relu', padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = Conv2D(512, (3,3),activation='relu', padding='same')(x)
#x = Conv2D(512, (3,3),activation='relu', padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)


x = Reshape([8*8, 512])(x) #x.shape[0] = 8?
att = multi_head(l=8*8, d=512 , dv=8*8, nv = 8)
x = att(x)
x = Dense(32, activation = 'relu')(x)
x = Flatten()(x)
out = Dense(6, activation='relu')(x)
model = Model(inputs = inp, outputs = out)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 128)       0   

In [13]:
h5_set_reader = h5py.File(train_set_file, 'r')
h5_GT_reader = h5py.File(train_GT_file, 'r')

print(list(h5_set_reader.keys()))
print(list(h5_GT_reader.keys()))

['train_A', 'train_B']
['train_A_labels', 'train_B_labels']


In [14]:
y = np.array(h5_GT_reader['train_A_labels'], dtype=np.float32, copy=False) 
split_idx = int(y.shape[0] * 0.9)

x_train = np.array(h5_set_reader['train_A'][:split_idx], dtype=np.float32, copy=False) / 255
x_val = np.array(h5_set_reader['train_A'][split_idx:], dtype=np.float32, copy=False) / 255
y_train = y[:split_idx, :]
y_val = y[split_idx:, :]

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
h5_set_reader.close()
h5_GT_reader.close()

(31178, 128, 128)
(31178, 6)
(3465, 128, 128)
(3465, 6)


In [15]:
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)
# x_train = np.stack((x_train, x_train, x_train), axis=-1)
# x_val = np.stack((x_val, x_val, x_val), axis=-1)
print(x_train.shape)
print(x_val.shape)

(31178, 128, 128, 1)
(3465, 128, 128, 1)


In [16]:
epochs = 100
batch_size = 1
epochs_to_wait_for_improve = 3
steps_per_epoch = x_train.shape[0] // batch_size
print(steps_per_epoch)

31178


In [0]:
#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
lr = 0.0001
momentum = 0
nesterov = False
optimizer = SGD(lr=lr, momentum=momentum, nesterov=nesterov)
#optimizer = Adam()
model.compile(optimizer=optimizer, loss=euclidean_loss(beta=beta), metrics=[euclidean_dist, x_error, y_error, z_error])
early_stopping_callback = EarlyStopping(monitor='val_euclidean_dist', patience=epochs_to_wait_for_improve, restore_best_weights=True)

In [0]:
img_gen = ImageDataGenerator()

In [0]:
model.fit(x_train, y_train, epochs=100)
            # steps_per_epoch=steps_per_epoch), validation_data=(x_val, y_val),
            # epochs=epochs, callbacks=[early_stopping_callback])

Epoch 1/100
975/975 [==============================] - 57s 58ms/step - loss: 0.5160 - euclidean_dist: 0.0501 - x_error: 0.0458 - y_error: 0.0355 - z_error: 0.0415
Epoch 2/100
975/975 [==============================] - 57s 58ms/step - loss: 0.4249 - euclidean_dist: 0.0279 - x_error: 0.0251 - y_error: 0.0255 - z_error: 0.0248
Epoch 3/100
975/975 [==============================] - 57s 59ms/step - loss: 0.4239 - euclidean_dist: 0.0279 - x_error: 0.0251 - y_error: 0.0255 - z_error: 0.0247
Epoch 4/100
975/975 [==============================] - 57s 58ms/step - loss: 0.4226 - euclidean_dist: 0.0279 - x_error: 0.0252 - y_error: 0.0255 - z_error: 0.0247
Epoch 5/100
975/975 [==============================] - 57s 58ms/step - loss: 0.4274 - euclidean_dist: 0.0278 - x_error: 0.0251 - y_error: 0.0255 - z_error: 0.0247
Epoch 6/100
975/975 [==============================] - 57s 59ms/step - loss: 0.4228 - euclidean_dist: 0.0278 - x_error: 0.0251 - y_error: 0.0255 - z_error: 0.0247
Epoch 7/100
975/975 [=

In [0]:
model.save(weight_file)
print 'Saved trained model at %s ' % weight_file

In [0]:
h5_reader = h5py.File(test_set_file, 'r')
print(list(h5_reader.keys()))
x_test = np.array(h5_reader['test'], dtype=np.float32, copy=False) / 255
print(x_test.shape)
x_test = np.stack((x_test, x_test, x_test), axis=-1)
h5_reader.close()

In [0]:
y_test = np.loadtxt(test_GT_file, dtype=np.float32)

In [0]:
print(x_test.shape)

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
metric_names = model.metrics_names

In [0]:
for metric, val in zip(metric_names, scores):
  print("{} : {}".format(metric, val))

In [0]:
file_obj = open(info_file, 'w')
file_obj.write("MODEL SUMMARY\n")
#summary = model.summary()
model.summary(print_fn=lambda x: file_obj.write(x + '\n'))
file_obj.write("\n")
#file_obj.write("Optimizer : Adam - batch_size = {}\n\n".format(batch_size))
file_obj.write("Optimizer : SGD - lr = {}, momentum = {}, batch_size = {}\n\n".format(lr, momentum, 32))

file_obj.write("\n\nPerformance")
for metric, val in zip(metric_names, scores):
  file_obj.write("{} : {}\n".format(metric, val))

file_obj.write("\n\nSpecial Comments : Used single channel gray image")

file_obj.write("\n\nFILE PATHS\n")
file_obj.write("Training set : {}\n".format(train_set_file))
file_obj.write("Testing set : {}\n".format(test_set_file))
file_obj.write("Weight file : {}\n".format(weight_file))
file_obj.close()